# Notebook Imports

In [1]:
import pandas as pd
import numpy as np

# Constants

In [2]:
TRAINING_DATA_FILE = 'SpamData/Training/train.txt'
TEST_DATA_FILE = 'SpamData/Testing/test.txt'

TOKEN_SPAM_PROB_FILE = 'SpamData/Testing_1/prob-spam.txt'
TOKEN_HAM_PROB_FILE = 'SpamData/Testing_1/prob-ham.txt'
TOKEN_ALL_PROB_FILE = 'SpamData/Testing_1/prob-ALL.txt'

TEST_FEATURE_MATRIX = 'SpamData/Testing_1/test-features.txt'
TEST_TARGET_FILE = 'SpamData/Testing_1/test-target.txt'

VOCAB_SIZE = 2500

In [3]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter=' ', dtype=int)

In [4]:
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter=' ', dtype=int)

In [5]:
sparse_train_data[:5]

array([[ 0,  0,  1,  2],
       [ 0,  2,  1,  1],
       [ 0, 18,  1,  1],
       [ 0, 19,  1,  1],
       [ 0, 26,  1,  2]])

In [6]:
sparse_train_data[-5:]

array([[5795, 2107,    0,    1],
       [5795, 2193,    0,    1],
       [5795, 2290,    0,    2],
       [5795, 2426,    0,    1],
       [5795, 2442,    0,    1]])

In [7]:
print('Number of rows in training file', sparse_train_data.shape[0])
print('Number of rows in test file', sparse_test_data.shape[0])

Number of rows in training file 260283
Number of rows in test file 113773


In [8]:
print('Number of emails in training file', np.unique(sparse_train_data[:, 0]).size)

Number of emails in training file 4012


In [9]:
print('Number of emails in test file', np.unique(sparse_test_data[:, 0]).size)

Number of emails in test file 1725


In [10]:
column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
column_names[:5]

['DOC_ID', 'CATEGORY', 0, 1, 2]

In [11]:
len(column_names)

2502

In [12]:
index_names = np.unique(sparse_train_data[:, 0])
index_names

array([   0,    1,    2, ..., 5791, 5794, 5795])

In [13]:
full_train_data = pd.DataFrame(index=index_names, columns=column_names)
full_train_data.fillna(value=0, inplace=True)

In [14]:
full_train_data.head()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
sparse_train_data[10:13]

array([[  0, 114,   1,   1],
       [  0, 118,   1,   1],
       [  0, 138,   1,   1]])

In [16]:
sparse_train_data[10][3]

1

In [17]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
    """
    Form a full matrix from a sparse matrix. Return a pandas dataframe. 
    Keyword arguments:
    sparse_matrix -- numpy array
    nr_words -- size of the vocabulary. Total number of tokens. 
    doc_idx -- position of the document id in the sparse matrix. Default: 1st column
    word_idx -- position of the word id in the sparse matrix. Default: 2nd column
    cat_idx -- position of the label (spam is 1, nonspam is 0). Default: 3rd column
    freq_idx -- position of occurrence of word in sparse matrix. Default: 4th column
    """
    column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
    doc_id_names = np.unique(sparse_matrix[:, 0])
    full_matrix = pd.DataFrame(index=doc_id_names, columns=column_names)
    full_matrix.fillna(value=0, inplace=True)
    
    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurrence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr, 'DOC_ID'] = doc_nr
        full_matrix.at[doc_nr, 'CATEGORY'] = label
        full_matrix.at[doc_nr, word_id] = occurrence
    
    full_matrix.set_index('DOC_ID', inplace=True)
    return full_matrix
    

In [18]:
%%time
full_train_data = make_full_matrix(sparse_train_data, VOCAB_SIZE)

CPU times: user 7.67 s, sys: 113 ms, total: 7.79 s
Wall time: 7.72 s


In [19]:
full_train_data.tail()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
5789,0,2,2,2,5,2,2,2,0,0,...,0,0,0,0,0,0,0,0,0,0
5790,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,0,2,0,2,0,1,0,2,0,1,...,0,0,0,0,0,0,0,0,0,0
5794,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5795,0,7,3,4,4,0,6,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [20]:
full_train_data.CATEGORY.size

4012

In [21]:
full_train_data.CATEGORY.sum()

1247

In [22]:
prob_spam = full_train_data.CATEGORY.sum() / full_train_data.CATEGORY.size
print('Probability of spam is', prob_spam)

Probability of spam is 0.3108175473579262


## Total Number of Words / Tokens

In [24]:
full_train_features = full_train_data.loc[:, full_train_data.columns != 'CATEGORY']
full_train_features.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,2,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,4,0,2,4,0,1,11,0,1,...,0,0,0,0,0,0,0,0,0,0
3,2,0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,11,1,1,0,1,32,2,11,4,...,0,2,0,0,0,0,0,0,0,0


In [25]:
email_lengths = full_train_features.sum(axis=1)
email_lengths.shape

(4012,)

In [26]:
email_lengths[:5]

DOC_ID
0      35
1      68
2     298
3      46
4    1434
dtype: int64

In [27]:
total_wc = email_lengths.sum()
total_wc

429458

## Number of Tokens in Spam & Ham Emails

In [28]:
spam_lengths = email_lengths[full_train_data.CATEGORY == 1]
spam_lengths.shape

(1247,)

In [29]:
spam_wc = spam_lengths.sum()
spam_wc

179959

In [30]:
ham_lengths = email_lengths[full_train_data.CATEGORY == 0]
ham_lengths.shape

(2765,)

In [31]:
email_lengths.shape[0] - spam_lengths.shape[0] - ham_lengths.shape[0]

0

In [32]:
nonspam_wc = ham_lengths.sum()
nonspam_wc

249499

In [33]:
spam_wc + nonspam_wc - total_wc

0

In [34]:
print('Average number of words in spam emails {:.0f}'.format(spam_wc / spam_lengths.shape[0]))
print('Average number of words in ham emails {:.3f}'.format(nonspam_wc / ham_lengths.shape[0]))

Average number of words in spam emails 144
Average number of words in ham emails 90.235


In [35]:
full_train_features.shape

(4012, 2500)

In [36]:
train_spam_tokens = full_train_features.loc[full_train_data['CATEGORY'] == 1]
train_spam_tokens.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,2,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,4,0,2,4,0,1,11,0,1,...,0,0,0,0,0,0,0,0,0,0
3,2,0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,11,1,1,0,1,32,2,11,4,...,0,2,0,0,0,0,0,0,0,0


In [37]:
train_spam_tokens.tail()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
1885,0,0,0,0,1,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1887,0,0,0,0,2,0,0,3,0,1,...,0,0,0,0,0,0,0,0,0,0
1889,1,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1890,0,1,1,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1895,0,0,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
train_spam_tokens.shape

(1247, 2500)

In [39]:
summed_spam_tokens = train_spam_tokens.sum(axis=0)+1

In [40]:
summed_spam_tokens.shape

(2500,)

In [41]:
train_ham_tokens = full_train_features.loc[full_train_data['CATEGORY'] == 0]

In [42]:
summed_ham_tokens = train_ham_tokens.sum(axis=0)+1

In [43]:
summed_ham_tokens

0       5290
1       2528
2       2044
3        899
4       1665
        ... 
2495      31
2496      30
2497      13
2498      26
2499      35
Length: 2500, dtype: int64

In [44]:
prob_tokens_spam = summed_spam_tokens / (spam_wc + VOCAB_SIZE) #p(Token/spam)
prob_tokens_spam

0       0.012233
1       0.005289
2       0.007322
3       0.011087
4       0.007245
          ...   
2495    0.000016
2496    0.000027
2497    0.000060
2498    0.000055
2499    0.000005
Length: 2500, dtype: float64

In [45]:
prob_tokens_ham = summed_ham_tokens / (nonspam_wc + VOCAB_SIZE) #p(Token/ham)
prob_tokens_ham

0       0.020992
1       0.010032
2       0.008111
3       0.003567
4       0.006607
          ...   
2495    0.000123
2496    0.000119
2497    0.000052
2498    0.000103
2499    0.000139
Length: 2500, dtype: float64

In [46]:
prob_tokens_all = full_train_features.sum(axis=0)/total_wc
prob_tokens_all

0       0.017510
1       0.008129
2       0.007866
3       0.006799
4       0.006951
          ...   
2495    0.000075
2496    0.000077
2497    0.000051
2498    0.000079
2499    0.000079
Length: 2500, dtype: float64

In [47]:
prob_tokens_all.sum()

1.0

In [48]:
np.savetxt(TOKEN_SPAM_PROB_FILE,prob_tokens_spam)
np.savetxt(TOKEN_HAM_PROB_FILE,prob_tokens_ham)
np.savetxt(TOKEN_ALL_PROB_FILE,prob_tokens_all)

# Preparing testing data

In [49]:
%%time
full_test_data = make_full_matrix(sparse_train_data,VOCAB_SIZE)

CPU times: user 8.47 s, sys: 152 ms, total: 8.63 s
Wall time: 8.54 s


In [50]:
x_test = full_test_data.loc[:,full_test_data.columns != 'CATEGORY']
x_test

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,2,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,4,0,2,4,0,1,11,0,1,...,0,0,0,0,0,0,0,0,0,0
3,2,0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,11,1,1,0,1,32,2,11,4,...,0,2,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5789,2,2,2,5,2,2,2,0,0,2,...,0,0,0,0,0,0,0,0,0,0
5790,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,2,0,2,0,1,0,2,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [51]:
y_test = full_test_data.loc[:,full_test_data.columns == 'CATEGORY']
y_test

,CATEGORY
DOC_ID,
0,1
1,1
2,1
3,1
4,1
...,...
5789,0
5790,0
5791,0


In [52]:
np.savetxt(TEST_TARGET_FILE,y_test)
np.savetxt(TEST_FEATURE_MATRIX,x_test)